# Titanic tutorial
- EDA
- sklearn
- ann

# EDA

In [28]:
# plot 을 그릴 때, FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version 제거
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# heatmap 같은 경우, 데이터가 커지면 수치가 잘 안 보인다
# plot 사이즈를 키운다
# sns.set(rc={'figure.figsize':(10,8)})

In [29]:
df_train = pd.read_csv('titanic/train.csv')
df_test = pd.read_csv('titanic/test.csv')

In [30]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [32]:
women = df_train.loc[df_train.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [33]:
men = df_train.loc[df_train.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


# Tutorial Results
- 0.77511

In [34]:
from sklearn.ensemble import RandomForestClassifier

y = df_train["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(df_train[features])
X_test = pd.get_dummies(df_test[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


# EDA Start
- 데이터들을 카테고리화 한다
- one hot encoding 을 한다

In [35]:
# 데이터 별 가족수를 카테고리화 한다.

df_train["family"] = df_train.SibSp + df_train.Parch
df_test["family"] = df_test.SibSp + df_test.Parch

def change_family_category(x):
  if x < 2: return "Single"
  if x == 2: return "Couple"
  if x < 5: return "Small"
  if x >= 5: return "Large"

df_train.family = df_train.family.apply(change_family_category)
df_test.family = df_test.family.apply(change_family_category)
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Single
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Single
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Single
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Single
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Single
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Single
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Single
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Small
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Single


In [36]:
# 5세 미만의 아동들의 생존률
df_train_baby_survive = df_train.loc[:, ['Survived', 'Age']]
df_train_baby_survive.fillna(df_train.Age.median())
df_train_baby_survive['is_baby'] = df_train_baby_survive.Age.map(lambda age: 1 if age < 5 else 0)

df_train_baby_survive_test = df_test.loc[:, ['Age']]
df_train_baby_survive_test.fillna(df_train.Age.median())
df_train_baby_survive_test['is_baby'] = df_train_baby_survive_test.Age.map(lambda age: 1 if age < 5 else 0)

df_train_baby_survive.corr()

,Survived,Age,is_baby
Survived,1.000000,-0.077221,0.129801
Age,-0.077221,1.000000,-0.460580
is_baby,0.129801,-0.460580,1.000000


In [37]:
#is_baby one hot encoding
df_train["is_baby"] = df_train_baby_survive.is_baby.copy()
df_test["is_baby"] = df_train_baby_survive_test.is_baby.copy()

In [38]:
# get initial from names

def get_initial(name:str):
  return name.split(',')[1].split('.')[0].strip()
df_train.Name.apply(get_initial).unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [39]:
df_train_initial_survive = df_train.loc[:, ['Survived', 'Name']]
df_train_initial_survive['initial'] = df_train_initial_survive.Name.apply(get_initial)

for initial in df_train_initial_survive['initial'].unique():
  df_train_initial_survive[initial] = df_train_initial_survive['initial'].map(lambda x: 1 if x == initial else 0)

df_train_initial_survive.drop(columns=["initial", "Name"], inplace=True)  
df_train_initial_survive.corr()

,Survived,Mr,Mrs,Miss,Master,Don,Rev,Dr,Mme,Ms,Major,Lady,Sir,Mlle,Col,Capt,the Countess,Jonkheer
Survived,1.000000,-0.549199,0.339040,0.327093,0.085221,-0.026456,-0.064988,0.008185,0.042470,0.042470,0.011329,0.042470,0.042470,0.060095,0.011329,-0.026456,0.042470,-0.026456
Mr,-0.549199,1.000000,-0.474952,-0.595692,-0.254903,-0.039411,-0.096808,-0.104624,-0.039411,-0.039411,-0.055767,-0.039411,-0.039411,-0.055767,-0.055767,-0.039411,-0.039411,-0.039411
Mrs,0.339040,-0.474952,1.000000,-0.204670,-0.087580,-0.013541,-0.033262,-0.035947,-0.013541,-0.013541,-0.019160,-0.013541,-0.013541,-0.019160,-0.019160,-0.013541,-0.013541,-0.013541
Miss,0.327093,-0.595692,-0.204670,1.000000,-0.109844,-0.016983,-0.041717,-0.045085,-0.016983,-0.016983,-0.024031,-0.016983,-0.016983,-0.024031,-0.024031,-0.016983,-0.016983,-0.016983
Master,0.085221,-0.254903,-0.087580,-0.109844,1.000000,-0.007267,-0.017851,-0.019292,-0.007267,-0.007267,-0.010283,-0.007267,-0.007267,-0.010283,-0.010283,-0.007267,-0.007267,-0.007267
Don,-0.026456,-0.039411,-0.013541,-0.016983,-0.007267,1.000000,-0.002760,-0.002983,-0.001124,-0.001124,-0.001590,-0.001124,-0.001124,-0.001590,-0.001590,-0.001124,-0.001124,-0.001124
Rev,-0.064988,-0.096808,-0.033262,-0.041717,-0.017851,-0.002760,1.000000,-0.007327,-0.002760,-0.002760,-0.003905,-0.002760,-0.002760,-0.003905,-0.003905,-0.002760,-0.002760,-0.002760
Dr,0.008185,-0.104624,-0.035947,-0.045085,-0.019292,-0.002983,-0.007327,1.000000,-0.002983,-0.002983,-0.004221,-0.002983,-0.002983,-0.004221,-0.004221,-0.002983,-0.002983,-0.002983
Mme,0.042470,-0.039411,-0.013541,-0.016983,-0.007267,-0.001124,-0.002760,-0.002983,1.000000,-0.001124,-0.001590,-0.001124,-0.001124,-0.001590,-0.001590,-0.001124,-0.001124,-0.001124
Ms,0.042470,-0.039411,-0.013541,-0.016983,-0.007267,-0.001124,-0.002760,-0.002983,-0.001124,1.000000,-0.001590,-0.001124,-0.001124,-0.001590,-0.001590,-0.001124,-0.001124,-0.001124


In [40]:
df_train_initial_survive = df_train.loc[:, ['Survived', 'Name']]
df_train_initial_survive['initial'] = df_train_initial_survive.Name.apply(get_initial)

special_initials = ['Mr', 'Mrs', 'Miss']
df_train_initial_survive['initial'] = df_train_initial_survive['initial'].map(lambda x: x if x in special_initials else "Others")
df_train['initial'] = df_train_initial_survive['initial'].copy()


# get dummies 에서 해주고 있기 때문에 지웁니다.
# label incoder랑 같은 역할을 해준다.

# special_initials = ['Mr', 'Mrs', 'Miss']
# for initial in special_initials:
#   df_train_initial_survive[initial] = df_train_initial_survive['initial'].map(lambda x: 1 if x == initial else 0)


# df_train_initial_survive['Others'] = df_train_initial_survive.initial.map(lambda x: 0 if x in special_initials else 1)
# df_train_initial_survive.drop(columns=["initial", "Name"], inplace=True)
# df_train_initial_survive.corr()

In [41]:
# pd_dummies 에서 해주기 때문에 지웁니다.

# df_train['Mr'] = df_train_initial_survive.Mr.copy()
# df_train['Mrs'] = df_train_initial_survive.Mrs.copy()
# df_train['Miss'] = df_train_initial_survive.Miss.copy()
# df_train['Others'] = df_train_initial_survive.Others.copy()

In [42]:
df_test_initial_survive = df_test.loc[:, ['Name']]
df_test_initial_survive['initial'] = df_test_initial_survive.Name.apply(get_initial)

special_initials = ['Mr', 'Mrs', 'Miss']
df_test_initial_survive['initial'] = df_test_initial_survive['initial'].map(lambda x: x if x in special_initials else "Others")
df_test['initial'] = df_test_initial_survive['initial'].copy()

# 밑에서 pd.dummies 가 다 해주기 때문에 지웁니다.

# special_initials = ['Mr', 'Mrs', 'Miss']
# for initial in special_initials:
#   df_test_initial_survive[initial] = df_test_initial_survive['initial'].map(lambda x: 1 if x == initial else 0)


# df_test_initial_survive['Others'] = df_test_initial_survive.initial.map(lambda x: 0 if x in special_initials else 1)
# df_test_initial_survive.drop(columns=["initial", "Name"], inplace=True)

# df_test['Mr'] = df_test_initial_survive.Mr.copy()
# df_test['Mrs'] = df_test_initial_survive.Mrs.copy()
# df_test['Miss'] = df_test_initial_survive.Miss.copy()
# df_test['Others'] = df_test_initial_survive.Others.copy()

In [43]:
df_train.Pclass = df_train.Pclass.map(lambda x : f"Pclass_{x}")
df_test.Pclass = df_test.Pclass.map(lambda x : f"Pclass_{x}")

In [44]:
# The mode is the value that appears most often
df_train.Embarked.mode()[0]

'S'

In [45]:
# 일단 추가
df_train.Embarked.fillna(df_train.Embarked.mode()[0], inplace=True)
df_test.Embarked.fillna(df_test.Embarked.mode()[0], inplace=True)

In [46]:
# 일단 추가
df_train.Fare.fillna(df_train.Fare.median(), inplace=True)
df_test.Fare.fillna(df_test.Fare.median(), inplace=True)

# RainForest Tries

In [47]:
df_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'family', 'is_baby', 'initial'],
      dtype='object')

In [48]:
df_test_passenger_id = df_test.PassengerId.copy()

In [49]:
from sklearn.ensemble import RandomForestClassifier

y = df_train["Survived"]

features = ["Pclass", "Sex", "family", "is_baby", "initial", "Embarked"]
X = pd.get_dummies(df_train[features])
X_test = pd.get_dummies(df_test[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': predictions})
output.to_csv('submission2.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


- 테스트 submission 결과: 0.77751

- Embarked 를 추가하고 나서 오히려 미세하게 증가함.
- 데이터 컬럼의 양에 따라서 정확도가 올라갈 수 있구나 알 수 있음
- 데이터 분석이 미흡했구나 알 수 있었다.


KFold 적용

In [50]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

#features = ["Pclass", "Sex", "SibSp", "Parch"]

features = ["Pclass", "Sex", "family", "is_baby", "initial", "Embarked"]

df_train = pd.read_csv('titanic/train.csv')

X = pd.get_dummies(df_train[features]).iloc[:200]
y = df_train["Survived"].iloc[:200]  # Ensure y has the same number of samples as X

rf_model = RandomForestClassifier()

kf = KFold(n_splits=30)
accuracies = []
precisions = []
recalls = []
f1_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    rf_model.fit(X_train, y_train)
    predictions = rf_model.predict(X_test)
    
    accuracies.append(accuracy_score(y_test, predictions))
    precisions.append(precision_score(y_test, predictions))
    recalls.append(recall_score(y_test, predictions))
    f1_scores.append(f1_score(y_test, predictions))

    print(f"Confusion Matrix (Fold {len(accuracies)}):\n{confusion_matrix(y_test, predictions)}\n")

average_accuracy = sum(accuracies) / len(accuracies)
average_precision = sum(precisions) / len(precisions)
average_recall = sum(recalls) / len(recalls)
average_f1 = sum(f1_scores) / len(f1_scores)

print(f"Average Accuracy: {average_accuracy}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1 Score: {average_f1}")

KeyError: "['family', 'is_baby', 'initial'] not in index"

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

# 가정: X는 특성 데이터, y는 타겟 데이터
features = ["Pclass", "Sex", "SibSp", "Parch"]

df_train = pd.read_csv('titanic/train.csv')
df_test = pd.read_csv('titanic/test.csv')

X = pd.get_dummies(df_train[features]).iloc[:]
y = df_train["Survived"]

# 랜덤 포레스트 모델 초기화
rf_model = RandomForestClassifier()

# K-Fold 교차 검증 설정
# 배치 사이즈를 나누듯 한 세트를 테스트 세트로 하고 나머지 네개에 대해 학습을 진행.
# n_splits=5 전체 데이터를 5개의 세트로 나눔. - 1세트를 테스트 세트로 생각, 나머지 4개로는 학습
# 한개의 테스트 세트를 5번 진행
# 5번의 값의 평균을 일반화하여 정확도를 계산한다.
kf = KFold(n_splits=20)  # 5겹 교차 검증-모델이 잘 만들었는 지 검증

# 각 분할에 대한 정확도를 저장할 리스트
# 아래 for문에 대한 5번의 값이 모두 들어있음
accuracies = []
conf_matrices = []  # Confusion matrices 리스트

# K-Fold 교차 검증 수행
# for문 총 5번 돔
for train_index, test_index in kf.split(X):
    # 훈련 데이터와 테스트 데이터 분할
    # print(train_index, test_index)
    # X_train으로 학습, X_test로 테스트 진행
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    # y_train은 정답 데이터 1 or 0 (학습 데이터에 대한 정답 데이터가)
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # 모델 훈련
    #rf_model이 학습된 모델이 들어가 있음
    rf_model.fit(X_train, y_train)

    # 예측 및 정확도 계산
    # 모델이 위 학습데이터에 대한 예측값을 출력
    predictions = rf_model.predict(X_test)
    # 정답값과 모델이 예측한 값에 대한 비교
    accuracy = accuracy_score(y_test, predictions)
    # accuracy에 대한 for문 5번이 출력됨
    accuracies.append(accuracy)
     # 각 Fold에 대한 혼동 행렬(Confusion Matrix)을 계산하고 이를 리스트에 추가함
    # y_test(정답데이터)와 predictions(예측값)를 기반으로 혼동 행렬을 계산하여 그 사잇값을 행렬로 나타냄
    cm = confusion_matrix(y_test, predictions)
    # 계산된 혼동 행렬(cm)을 conf_matrices 리스트에 추가. 이는 각 Fold에 대한 혼동 행렬들을 저장.
    conf_matrices.append(cm)
    # 현재 Fold에 대한 혼동 행렬을 출력함. Fold 번호와 함께 혼동 행렬의 내용이 출력됨.
    print(f"Confusion Matrix (Fold {len(conf_matrices)}):\n{cm}\n")

# 평균 정확도 출력
average_accuracy = np.mean(accuracies)
print(f"Average Accuracy: {average_accuracy}")

Confusion Matrix (Fold 1):
[[5 0]
 [0 5]]

Confusion Matrix (Fold 2):
[[4 1]
 [1 4]]

Confusion Matrix (Fold 3):
[[4 1]
 [3 2]]

Confusion Matrix (Fold 4):
[[6 0]
 [2 2]]

Confusion Matrix (Fold 5):
[[4 3]
 [0 3]]

Confusion Matrix (Fold 6):
[[5 0]
 [1 4]]

Confusion Matrix (Fold 7):
[[6 0]
 [2 2]]

Confusion Matrix (Fold 8):
[[7 0]
 [2 1]]

Confusion Matrix (Fold 9):
[[5 0]
 [3 2]]

Confusion Matrix (Fold 10):
[[8 0]
 [1 1]]

Confusion Matrix (Fold 11):
[[6 1]
 [2 1]]

Confusion Matrix (Fold 12):
[[6 4]
 [0 0]]

Confusion Matrix (Fold 13):
[[6 0]
 [2 2]]

Confusion Matrix (Fold 14):
[[8 0]
 [0 2]]

Confusion Matrix (Fold 15):
[[3 4]
 [2 1]]

Confusion Matrix (Fold 16):
[[7 1]
 [0 2]]

Confusion Matrix (Fold 17):
[[6 1]
 [1 2]]

Confusion Matrix (Fold 18):
[[8 1]
 [0 1]]

Confusion Matrix (Fold 19):
[[6 0]
 [3 1]]

Confusion Matrix (Fold 20):
[[3 1]
 [2 4]]

Average Accuracy: 0.775


In [51]:
import pandas as pd
#from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
#from sklearn.ensemble import RandomForestClassifier
#import pandas as pd
#import numpy as np

# 가정: X는 특성 데이터, y는 타겟 데이터
features = ["Pclass", "Sex", "SibSp", "Parch"]


X = pd.get_dummies(df_train[features]).iloc[:]
y = df_train["Survived"]

rf_model = RandomForestClassifier()
kf = KFold(n_splits=20)

accuracies = []
conf_matrices = []  # Confusion matrices 리스트: 분류 모델의 성능을 평가

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    rf_model.fit(X_train, y_train)
    predictions = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    accuracies.append(accuracy)

    # Calculate confusion matrix for each fold
    # 각 Fold에 대한 혼동 행렬(Confusion Matrix)을 계산하고 이를 리스트에 추가함
    # y_test(정답데이터)와 predictions(예측값)를 기반으로 혼동 행렬을 계산하여 그 사잇값을 행렬로 나타냄
    cm = confusion_matrix(y_test, predictions)
    # 계산된 혼동 행렬(cm)을 conf_matrices 리스트에 추가. 이는 각 Fold에 대한 혼동 행렬들을 저장.
    conf_matrices.append(cm)
    # 현재 Fold에 대한 혼동 행렬을 출력함. Fold 번호와 함께 혼동 행렬의 내용이 출력됨.
    print(f"Confusion Matrix (Fold {len(conf_matrices)}):\n{cm}\n")

average_accuracy = np.mean(accuracies)
print(f"Average Accuracy: {average_accuracy}")

Confusion Matrix (Fold 1):
[[18  6]
 [ 6 15]]

Confusion Matrix (Fold 2):
[[26  1]
 [ 7 11]]

Confusion Matrix (Fold 3):
[[30  5]
 [ 5  5]]

Confusion Matrix (Fold 4):
[[29  6]
 [ 3  7]]

Confusion Matrix (Fold 5):
[[22  3]
 [10 10]]

Confusion Matrix (Fold 6):
[[22  7]
 [ 5 11]]

Confusion Matrix (Fold 7):
[[18  5]
 [ 7 15]]

Confusion Matrix (Fold 8):
[[21  1]
 [ 5 18]]

Confusion Matrix (Fold 9):
[[22  5]
 [ 3 15]]

Confusion Matrix (Fold 10):
[[20  4]
 [10 11]]

Confusion Matrix (Fold 11):
[[31  1]
 [ 6  7]]

Confusion Matrix (Fold 12):
[[21  6]
 [ 4 13]]

Confusion Matrix (Fold 13):
[[19  3]
 [ 9 13]]

Confusion Matrix (Fold 14):
[[26  3]
 [ 6  9]]

Confusion Matrix (Fold 15):
[[24  4]
 [ 7  9]]

Confusion Matrix (Fold 16):
[[23  4]
 [ 9  8]]

Confusion Matrix (Fold 17):
[[27  1]
 [ 6 10]]

Confusion Matrix (Fold 18):
[[26  3]
 [ 3 12]]

Confusion Matrix (Fold 19):
[[26  3]
 [ 8  7]]

Confusion Matrix (Fold 20):
[[24  3]
 [ 5 12]]

Average Accuracy: 0.7778030303030303


# ANN Tries
- use the same data as above

In [ ]:
X.shape

(891, 5)

In [ ]:
X_test.shape

(44, 5)

In [ ]:
pd.get_dummies(df_train[features])

,Pclass,SibSp,Parch,Sex_female,Sex_male
0,3,1,0,False,True
1,1,1,0,True,False
2,3,0,0,True,False
3,1,1,0,True,False
4,3,0,0,False,True
...,...,...,...,...,...
886,2,0,0,False,True
887,1,0,0,True,False
888,3,1,2,True,False
889,1,0,0,False,True


In [ ]:
from torch.utils import data

In [ ]:
training_dataloader = data.DataLoader(pd.get_dummies(df_train[features]), shuffle=True, batch_size=100)

In [ ]:
from utils import train, test
from torch import nn

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(nn_model.parameters(), lr=learning_rate)

NameError: name 'optim' is not defined

In [ ]:
train(training_dataloader, nn_model, loss, optimizer)

KeyError: 125

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

one_hot_encoder = OneHotEncoder()
X_train = one_hot_encoder.fit_transform(X).toarray()
X_test = one_hot_encoder.fit_transform(X_test).toarray()

In [ ]:
X_train.shape

(891, 34)

In [ ]:
X_test.shape

(418, 34)

In [ ]:
X_test

array([[1., 0., 1., ..., 1., 1., 0.],
       [1., 0., 1., ..., 0., 0., 1.],
       [1., 0., 1., ..., 1., 1., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 1., 0.]])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X_train, y.array, test_size = 0.1)

In [ ]:
from sklearn.utils import shuffle
from torch.autograd import Variable
from torch import FloatTensor, LongTensor, nn, optim
import torch

import numpy as np

from model import nn_model

In [ ]:
batch_size = 50
num_epochs = 50
learning_rate = 0.01
batch_no = len(x_train) // batch_size



for epoch in range(num_epochs):
    if epoch % 5 == 0:
        print('Epoch {}'.format(epoch+1))
    x_train, y_train = shuffle(x_train, y_train)
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        x_var = Variable(FloatTensor(x_train[start:end]))
        y_var = Variable(LongTensor(y_train[start:end]))
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        ypred_var = nn_model(x_var)
        loss =criterion(ypred_var, y_var)
        loss.backward()
        optimizer.step()

Epoch 1


/home/sdk/miniconda3/envs/MathAI/lib/python3.10/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46


In [ ]:
test_var = Variable(FloatTensor(x_val), requires_grad=True)
with torch.no_grad():
    result = nn_model(test_var)
values, labels = torch.max(result, 1)
num_right = np.sum(labels.data.numpy() == y_val)
print('Accuracy {:.2f}'.format(num_right / len(y_val)))

Accuracy 0.79


In [ ]:
X_test_var = Variable(FloatTensor(X_test), requires_grad=True)
with torch.no_grad():
    result = nn_model(X_test_var)
values, labels = torch.max(result, 1)
survived = labels.data.numpy()

In [ ]:
X_test_passenger_id = df_test['PassengerId']

In [ ]:
import csv

submission = [['PassengerId', 'Survived']]
for i in range(len(survived)):
    submission.append([X_test_passenger_id[i], survived[i]])

with open('submission.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerows(submission)
    
print('Writing Complete!')

Writing Complete!


In [ ]:
77033